In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1

original_model = load_model('mnist_model.h5')
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
new_model = tf.keras.Sequential()

for layer in original_model.layers:
    if isinstance(layer, Dense):
        pruned_layer = tf.keras.layers.Dense(
            units=layer.units,
            activation=layer.activation,
            kernel_initializer=layer.kernel_initializer,
            kernel_regularizer=l1(0.01),  
        )
        new_model.add(pruned_layer)
    else:
        new_model.add(layer)

new_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

new_model.summary()

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
new_model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])
new_model.save('SparseNeuralNet.h5')
# Evaluate the pruned model
loss, accuracy = new_model.evaluate(x_test, y_test)
print(f'Pruned Model - Loss: {loss}, Accuracy: {accuracy}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/50


ValueError: in user code:

    File "c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\bansa\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\bansa\AppDa